**This notebook contans the gradio app for displaying yolo v8 model results**

In [1]:
!pip install ultralytics
!pip install -q gradio
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.138 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.4/78.2 GB disk)


In [2]:
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg

--2023-07-20 09:03:24--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 271475 (265K) [image/jpeg]
Saving to: ‘images/truck.jpg’

truck.jpg           100%[===================>] 265.11K  --.-KB/s    in 0.004s  

2023-07-20 09:03:24 (67.4 MB/s) - ‘images/truck.jpg’ saved [271475/271475]



In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
from moviepy.editor import *
from google.colab.patches import cv2_imshow
import moviepy.editor as mp
import gradio as gr

In [ ]:
def process_video(input_video):

  # Step 2: Extract frames
  video = cv2.VideoCapture(input_video)
  fps = video.get(cv2.CAP_PROP_FPS)
  frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  output_video = "/content/gdrive/My Drive/demoM/test_images/output_video.mp4"
  output_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))

  # Step 3: Pass each frame to a model to detect objects
  model = YOLO("/content/gdrive/My Drive/demoM/trainedModel/bestV1.pt")

  # Step 4: Extract audio from the input video
  video_clip = mp.VideoFileClip(input_video)
  audio_clip = video_clip.audio
  audio_clip.write_audiofile("temp_audio.wav")

  while True:
      # Read the next frame
      try:
        ret, frame = video.read()
        if not ret:
            break
        # Step 5: Get the mask of the object and blur the mask
        results = model.predict(source=frame, conf=0.5)
        objects_mask = results[0].masks.xy
        mask_ = np.zeros_like(frame)
        for mask in objects_mask:
          vertices = mask.reshape(mask.shape[0], mask.shape[1] // 2, 2)
          cv2.fillPoly(mask_, np.int32([vertices]), (255, 255, 255))
          #blurred_region1 = cv2.blur(frame, (20, 20), 10)
          blurred_region2 = cv2.GaussianBlur(frame, (53, 53), 30)
          output_frame = np.where(mask_ != 0, blurred_region2, frame)
        output_writer.write(output_frame)
      except Exception as e:
        output_writer.write(frame)

  # Release the video capture and writer, and close windows
  video.release()
  output_writer.release()

  # Step 8: Merge the output video with the audio using moviepy
  final_output = mp.VideoFileClip(output_video)
  final_output = final_output.set_audio(mp.AudioFileClip("temp_audio.wav"))
  final_output.write_videofile("/content/gdrive/My Drive/demoM/test_images/final_output_video.mp4", codec="libx264", audio_codec="aac", remove_temp=True)
  return "/content/gdrive/My Drive/demoM/test_images/final_output_video.mp4"


# Define the input and output components
input_video = gr.inputs.Video(type="mp4", label="Input Video")
output_video = gr.outputs.File(label="Processed Video")

# Create the Gradio interface
interface = gr.Interface(fn=process_video, inputs=input_video, outputs=output_video)

# Launch the interface
interface.launch(share=True)